In [1]:
import neo
from networkunit import tests, scores, models
#from networkunit.capabilities.cap_ProducesSpikeTrains import ProducesSpikeTrains
from networkunit.capabilities.ProducesSpikeTrains import ProducesSpikeTrains # branch v0.2
from copy import copy
import pandas as pd
import numpy as np
import elephant
import quantities as pq
import sciunit
from quantities import ms
import sys
sys.setrecursionlimit(10000)
from pathlib import Path

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\parallel\__init__.py:47: UserWarning: mpi4py package is missing. Please run 'pip install mpi4py' in a terminal to activate MPI features.
  warnings.warn("mpi4py package is missing. Please run 'pip install mpi4py' "


In [11]:
import elephant

In [12]:
elephant.__version__

'0.10.0'

We will create a class for each seed value for each res for each model (droop and equal), then run wasserstein metric, and possibly others, one-vs-all, for all seeds one case against continuous model seed 1. Then run the metric for continuous different seeds against each other to get a view of what distance is still considered same class.

In [2]:
def create_class(path):
    """
    This function creates a model class for the specific file path given.
    """
    class model(models.loaded_spiketrains): 
    
        default_params = {'file_path':path}

        def load(self):
            file_path = self.default_params['file_path']        
            data = pd.read_csv(file_path,skiprows=2,sep='\t')
            spike_data = data.sort_values(by='time_ms')
            grouped = spike_data.groupby(spike_data['sender'])

            spike_trains = []
            for name, group in grouped:
                    """
                    Each group is senders and times for one value of senders. That is, we iterate through all 
                    neurons. And the times for each neuron is in sorted order. Therefore, the cvs
                    returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
                    """
                    t = np.asarray(group['time_ms'])
                    spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
                    spike_trains.append(spiketrain)
             # make sure we get spiketrains of length 1065 for microcircuit
            
            if len(spike_trains) < 1065:
                n = 1065 - len(spike_trains)
                t = np.asarray([0])
                for i in range(n):
                    sp = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
                    spike_trains.append(sp)
            return spike_trains
    return model

In [3]:
def get_classes(path_list):
    classes = []
    for p in path_list:
        cl = create_class(p)
        classes.append(cl)
    return classes

In [4]:
class FR_test_class(sciunit.TestM2M, tests.firing_rate_test):
    """
    This finds the wasserstein distance based on firing rates.
    """
    score_type = scores.wasserstein_distance
    
FR_test = FR_test_class()

In [7]:
class CV_test_class(sciunit.TestM2M, tests.isi_variation_test):
    """
    This finds the wasserstein distance based on CVs.
    """
    default_params = {'variation_measure': 'cv'} # need to set cv as lvr is default in tests.isi_variation_test
    score_type = scores.wasserstein_distance
    
CV_test = CV_test_class()

In [9]:
class CC_test_class(sciunit.TestM2M, tests.correlation_matrix_test):
    """
    This finds the wasserstein distance based on CC matrices.
    """
    score_type = scores.wasserstein_distance
    
CC_test = CC_test_class()

We will start by doing this for the Brunel model.

In [5]:
cont_seed_1 = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_continuous\\brunel_continuous_delay_1.0_2.0_seed_1_spikes_exc-12502-0.dat'
cont_1 = create_class(cont_seed_1)
# create instance of class
cont = cont_1()
# produce spike trains
cont.produce_spiketrains();

File paths to all droop models, organized in lists so that all files with same resolution is in one list.

In [7]:
droop_1_2 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_2_classes = get_classes(droop_1_2)
droop_1_2_instances = create_instances(droop_1_2_classes)


In [8]:
FR_scores = FR_test.judge([cont,droop_1_2_instances[0]])

In [9]:
v = FR_scores.score.iloc[1,0]

In [10]:
v

1.3170816898345947

In [6]:
droop_1_2 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_4 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_4\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_8 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_16 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_16\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_32 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_32\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_64 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_64\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_128 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_128\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
droop_1_256 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_256\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]

Get the classes in a list

In [7]:
droop_1_2_classes = get_classes(droop_1_2)
droop_1_4_classes = get_classes(droop_1_4)
droop_1_8_classes = get_classes(droop_1_8)
droop_1_16_classes = get_classes(droop_1_16)
droop_1_32_classes = get_classes(droop_1_32)
droop_1_64_classes = get_classes(droop_1_64)
droop_1_128_classes = get_classes(droop_1_128)
droop_1_256_classes = get_classes(droop_1_256)

Create instance of each class and run create_spiketrains() for each class and test it against continuous model.

In [6]:
def create_instances(class_list):
    """
    Iterate through list of classes, create instance and run create_spiketrains(). Return list of the instances, 
    ready to be measured.
    """
    instances = []
    for cl in class_list:
        c = cl()
        c.produce_spiketrains();
        instances.append(c)
    return instances

In [9]:
droop_1_2_instances = create_instances(droop_1_2_classes)
droop_1_4_instances = create_instances(droop_1_4_classes)
droop_1_8_instances = create_instances(droop_1_8_classes)
droop_1_16_instances = create_instances(droop_1_16_classes)
droop_1_32_instances = create_instances(droop_1_32_classes)
droop_1_64_instances = create_instances(droop_1_64_classes)
droop_1_128_instances = create_instances(droop_1_128_classes)
droop_1_256_instances = create_instances(droop_1_256_classes)

In [10]:
def get_ws_distances(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,10):
        FR_scores = FR_test.judge([cont,list_of_instances[i]])
        v = FR_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [11]:
ws_droop_1_2 = get_ws_distances(droop_1_2_instances)

ws_droop_1_4 = get_ws_distances(droop_1_4_instances)

ws_droop_1_8 = get_ws_distances(droop_1_8_instances)

ws_droop_1_16 = get_ws_distances(droop_1_16_instances)

ws_droop_1_32 = get_ws_distances(droop_1_32_instances)

ws_droop_1_64 = get_ws_distances(droop_1_64_instances)

ws_droop_1_128 = get_ws_distances(droop_1_128_instances)

ws_droop_1_256 = get_ws_distances(droop_1_256_instances)

KeyboardInterrupt: 

Now for equal model against cont seed 1.

In [15]:
ws_droop_1_2 = [1,2,3,4,1,2,3,4,1,1]

ws_droop_1_4 = [5,6,7,8,1,2,3,4,1,1]

ws_droop_1_8 = [9,10,11,12,1,2,3,4,1,1]

ws_droop_1_16 = [13,14,15,16,1,2,3,4,1,1]

ws_droop_1_32 = [17,18,19,20,1,2,3,4,1,1]

ws_droop_1_64 = [21,22,23,24,1,2,3,4,1,1]

ws_droop_1_128 =[25,26,27,28,1,2,3,4,1,1]

ws_droop_1_256 = [29,30,31,32,1,2,3,4,1,1]

In [18]:
ws_droop_1_2 = [1,2,3,4,1,2,3,4,1,1]

In [ ]:
equal_1_2 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_equal\\brunel_rounding_True_delay_0.75_2.25_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_4 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_4\\brunel_rounding_equal\\brunel_rounding_True_delay_0.875_2.125_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_8 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_rounding_equal\\brunel_rounding_True_delay_0.9375_2.0625_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_16 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_16\\brunel_rounding_equal\\brunel_rounding_True_delay_0.96875_2.03125_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_32 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_32\\brunel_rounding_equal\\brunel_rounding_True_delay_0.984375_2.015625_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_64 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_64\\brunel_rounding_equal\\brunel_rounding_True_delay_0.9921875_2.0078125_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_128 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_128\\brunel_rounding_equal\\brunel_rounding_True_delay_0.99609375_2.00390625_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]
equal_1_256 = [f'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_256\\brunel_rounding_equal\\brunel_rounding_True_delay_0.998046875_2.001953125_seed_{i}_spikes_exc-12502-0.dat'.format(i) for i in range(1,11)]

In [ ]:
equal_1_2_classes = get_classes(equal_1_2)
equal_1_4_classes = get_classes(equal_1_4)
equal_1_8_classes = get_classes(equal_1_8)
equal_1_16_classes = get_classes(equal_1_16)
equal_1_32_classes = get_classes(equal_1_32)
equal_1_64_classes = get_classes(equal_1_64)
equal_1_128_classes = get_classes(equal_1_128)
equal_1_256_classes = get_classes(equal_1_256)

In [ ]:
equal_1_2_instances = create_instances(equal_1_2_classes)
equal_1_4_instances = create_instances(equal_1_4_classes)
equal_1_8_instances = create_instances(equal_1_8_classes)
equal_1_16_instances = create_instances(equal_1_16_classes)
equal_1_32_instances = create_instances(equal_1_32_classes)
equal_1_64_instances = create_instances(equal_1_64_classes)
equal_1_128_instances = create_instances(equal_1_128_classes)
equal_1_256_instances = create_instances(equal_1_256_classes)

# Microcircuit
Do the same for microcircuit model, but in this case we gathered all spike times for each case in files.

In [2]:
base = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\microcircuit_spike_data'
droop_1_2 = [f'{base}\\droop\\res_1_2\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_4 = [f'{base}\\droop\\res_1_4\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_8 = [f'{base}\\droop\\res_1_8\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_16 = [f'{base}\\droop\\res_1_16\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_32 = [f'{base}\\droop\\res_1_32\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_64 = [f'{base}\\droop\\res_1_64\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_128 = [f'{base}\\droop\\res_1_128\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
droop_1_256 = [f'{base}\\droop\\res_1_256\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]

In [3]:
equal_1_8 = [f'{base}\\equal\\res_1_8\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
equal_1_16 = [f'{base}\\equal\\res_1_16\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
equal_1_32 = [f'{base}\\equal\\res_1_32\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
equal_1_64 = [f'{base}\\equal\\res_1_64\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
equal_1_128 = [f'{base}\\equal\\res_1_128\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]
equal_1_256 = [f'{base}\\equal\\res_1_256\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]

In [4]:
cont = [f'{base}\\continuous\\res_1_8\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(1,11)]

In [22]:
def create_class(path):
    """
    This function creates a model class for the specific file path given.
    
        
    There will be errors if the length of the spike trains list is less than that of the continuous one (1065) when calculating
    WS distances later on. Therefore if the length is less, fill in spiketrains of 0's. 
    """
    class model(models.loaded_spiketrains): 
    
        default_params = {'file_path':path}

        def load(self):
            file_path = self.default_params['file_path']        
            data = pd.read_pickle(file_path)
            spike_data = data.sort_values(by='time_ms')
            grouped = spike_data.groupby(spike_data['sender'])

            spike_trains = []
            for name, group in grouped:
                    """
                    Each group is senders and times for one value of senders. That is, we iterate through all 
                    neurons. And the times for each neuron is in sorted order. Therefore, the cvs
                    returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
                    """
                    t = np.asarray(group['time_ms'])
                    spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
                    spike_trains.append(spiketrain)
            # make sure we get spiketrains of length 1065 for microcircuit
            
            if len(spike_trains) < 1065:
                n = 1065 - len(spike_trains)
                t = np.asarray([0])
                for i in range(n):
                    sp = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
                    spike_trains.append(sp)
            return spike_trains
    return model

In [23]:
def get_classes(path_list):
    classes = []
    for p in path_list:
        cl = create_class(p)
        classes.append(cl)
    return classes

In [24]:
class FR_test_class(sciunit.TestM2M, tests.firing_rate_test):
    score_type = scores.wasserstein_distance
    
FR_test = FR_test_class()

In [25]:
class CV_test_class(sciunit.TestM2M, tests.isi_variation_test):
    """
    This finds the wasserstein distance based on CVs.
    """
    default_params = {'variation_measure': 'cv'} # need to set cv as lvr is default in tests.isi_variation_test
    score_type = scores.wasserstein_distance
    
CV_test = CV_test_class()

In [26]:
class CC_test_class(sciunit.TestM2M, tests.correlation_matrix_test):
    """
    This finds the wasserstein distance based on CC matrices.
    """
    score_type = scores.wasserstein_distance
    
CC_test = CC_test_class()

In [27]:
def create_instances(class_list):
    """
    Iterate through list of classes, create instance and run create_spiketrains(). Return list of the instances, 
    ready to be measured.
    """
    instances = []
    for cl in class_list:
        c = cl()
        c.produce_spiketrains();
        instances.append(c)
    return instances

In [28]:
def get_ws_distances(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,10):
        FR_scores = FR_test.judge([cont,list_of_instances[i]])
        v = FR_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [29]:
def get_ws_distances_cv(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,10):
        CV_scores = CV_test.judge([cont,list_of_instances[i]])
        v = CV_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [30]:
def get_ws_distances_cc(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,10):
        CC_scores = CC_test.judge([cont,list_of_instances[i]])
        v = CC_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

Remember, we will only compare the models for resolution 1/8 and up as the activity in equal model is weird for res 1/4 and it does not exist for 1/2. So maybe we only look at these.

In [31]:
droop_1_8_classes = get_classes(droop_1_8)
droop_1_16_classes = get_classes(droop_1_16)
droop_1_32_classes = get_classes(droop_1_32)
droop_1_64_classes = get_classes(droop_1_64)
droop_1_128_classes = get_classes(droop_1_128)
droop_1_256_classes = get_classes(droop_1_256)

In [32]:
droop_1_8_instances = create_instances(droop_1_8_classes)
droop_1_16_instances = create_instances(droop_1_16_classes)
droop_1_32_instances = create_instances(droop_1_32_classes)
droop_1_64_instances = create_instances(droop_1_64_classes)
droop_1_128_instances = create_instances(droop_1_128_classes)
droop_1_256_instances = create_instances(droop_1_256_classes)

In [33]:
cont_seed_1 = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\microcircuit_spike_data\\continuous\\res_1_8\\seed_1\\spike_data.pkl'
cont_1 = create_class(cont_seed_1)
# create instance of class
cont = cont_1()
# produce spike trains
cont.produce_spiketrains();

In [33]:
ws_droop_1_8 = get_ws_distances(droop_1_8_instances)

ws_droop_1_16 = get_ws_distances(droop_1_16_instances)

ws_droop_1_32 = get_ws_distances(droop_1_32_instances)

ws_droop_1_64 = get_ws_distances(droop_1_64_instances)

ws_droop_1_128 = get_ws_distances(droop_1_128_instances)

ws_droop_1_256 = get_ws_distances(droop_1_256_instances)

In [34]:
ws_cv_droop_1_8 = get_ws_distances_cv(droop_1_8_instances)

ws_cv_droop_1_16 = get_ws_distances_cv(droop_1_16_instances)

ws_cv_droop_1_32 = get_ws_distances_cv(droop_1_32_instances)

ws_cv_droop_1_64 = get_ws_distances_cv(droop_1_64_instances)

ws_cv_droop_1_128 = get_ws_distances_cv(droop_1_128_instances)

ws_cv_droop_1_256 = get_ws_distances_cv(droop_1_256_instances)

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
ws_cc_droop_1_8 = get_ws_distances_cc(droop_1_8_instances)

ws_cc_droop_1_16 = get_ws_distances_cc(droop_1_16_instances)

ws_cc_droop_1_32 = get_ws_distances_cc(droop_1_32_instances)

ws_cc_droop_1_64 = get_ws_distances_cc(droop_1_64_instances)

ws_cc_droop_1_128 = get_ws_distances_cc(droop_1_128_instances)

ws_cc_droop_1_256 = get_ws_distances_cc(droop_1_256_instances)

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 2 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 3 last spike(s) of the input sp

In [60]:
t = pd.DataFrame(ws_cc_droop_1_256, index = [1,2,3,4,5,6,7,8,9,10])
t

0
1   36.204021
2   32.870064
3   25.781115
4   31.373013
5   28.166391
6   27.085144
7   28.175507
8   35.639111
9   28.581930
10  37.358109

In [61]:
put = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\master_thesis_code\\wasserstein\\distances'
t.to_csv(f'{put}\\ws_cc_microcircuit_droop_1_256.csv'.format(put))

In [62]:
equal_1_8_classes = get_classes(equal_1_8)
equal_1_16_classes = get_classes(equal_1_16)
equal_1_32_classes = get_classes(equal_1_32)
equal_1_64_classes = get_classes(equal_1_64)
equal_1_128_classes = get_classes(equal_1_128)
equal_1_256_classes = get_classes(equal_1_256)

In [63]:
equal_1_8_instances = create_instances(equal_1_8_classes)
equal_1_16_instances = create_instances(equal_1_16_classes)
equal_1_32_instances = create_instances(equal_1_32_classes)
equal_1_64_instances = create_instances(equal_1_64_classes)
equal_1_128_instances = create_instances(equal_1_128_classes)
equal_1_256_instances = create_instances(equal_1_256_classes)

In [64]:
ws_cv_equal_1_8 = get_ws_distances_cv(equal_1_8_instances)

ws_cv_equal_1_16 = get_ws_distances_cv(equal_1_16_instances)

ws_cv_equal_1_32 = get_ws_distances_cv(equal_1_32_instances)

ws_cv_equal_1_64 = get_ws_distances_cv(equal_1_64_instances)

ws_cv_equal_1_128 = get_ws_distances_cv(equal_1_128_instances)

ws_cv_equal_1_256 = get_ws_distances_cv(equal_1_256_instances)

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

In [75]:
t = pd.DataFrame(ws_cv_equal_1_256, index = [1,2,3,4,5,6,7,8,9,10])
t

0
1   0.045418
2   0.090050
3   0.085958
4   0.080916
5   0.433152
6   0.074328
7   0.087506
8   0.101961
9   0.520284
10  0.077228

In [76]:
put = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\master_thesis_code\\wasserstein\\distances'
t.to_csv(f'{put}\\ws_cv_microcircuit_equal_1_256.csv'.format(put))

In [77]:
ws_cc_equal_1_8 = get_ws_distances_cc(equal_1_8_instances)

ws_cc_equal_1_16 = get_ws_distances_cc(equal_1_16_instances)

ws_cc_equal_1_32 = get_ws_distances_cc(equal_1_32_instances)

ws_cc_equal_1_64 = get_ws_distances_cc(equal_1_64_instances)

ws_cc_equal_1_128 = get_ws_distances_cc(equal_1_128_instances)

ws_cc_equal_1_256 = get_ws_distances_cc(equal_1_256_instances)

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 7 last spike(s) of the input spiketrain
  warnings.warn("Binning discarded {} last spike(s) of the "
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\elephant\conversion.py:1168: UserWarning: Binning discarded 2 last spike(s) of the input sp

In [88]:
t = pd.DataFrame(ws_cc_equal_1_256, index = [1,2,3,4,5,6,7,8,9,10])
t

0
1   34.404720
2   27.106716
3   26.970499
4   29.101572
5   46.220165
6   26.861279
7   32.791573
8   30.131277
9   46.354202
10  29.966475

In [89]:
put = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\master_thesis_code\\wasserstein\\distances'
t.to_csv(f'{put}\\ws_cc_microcircuit_equal_1_256.csv'.format(put))

In [58]:
ws_equal_1_8 = get_ws_distances(equal_1_8_instances)

ws_equal_1_16 = get_ws_distances(equal_1_16_instances)

ws_equal_1_32 = get_ws_distances(equal_1_32_instances)

ws_equal_1_64 = get_ws_distances(equal_1_64_instances)

ws_equal_1_128 = get_ws_distances(equal_1_128_instances)

ws_equal_1_256 = get_ws_distances(equal_1_256_instances)

In [74]:
ws_equal_1_256

[0.05357803776860237,
 0.04543914645910263,
 0.027011744678020477,
 0.04007922485470772,
 0.1258789449930191,
 0.03597483038902283,
 0.0515923798084259,
 0.04547854885458946,
 0.12121787667274475,
 0.05763678625226021]

In [72]:
t = pd.DataFrame(ws_equal_1_128, index = [1,2,3,4,5,6,7,8,9,10])
t

0
1   0.017912
2   0.049209
3   0.039033
4   0.039051
5   0.068773
6   0.022194
7   0.044142
8   0.050234
9   0.027322
10  0.040473

In [73]:
put = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\master_thesis_code\\wasserstein\\distances'
t.to_csv(f'{put}\\ws_microcircuit_equal_1_128.csv'.format(put))

Continuous seeds 2,...,10 vs continuous seed 1

In [90]:
def get_ws_distances(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,9):
        FR_scores = FR_test.judge([cont,list_of_instances[i]])
        v = FR_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [91]:
def get_ws_distances_cv(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,9):
        CV_scores = CV_test.judge([cont,list_of_instances[i]])
        v = CV_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [92]:
def get_ws_distances_cc(list_of_instances):
    """
    Run the wasserstein metric on continuous model vs all seeds one model one resolution by passing list of 
    instances of class.
    """
    ws_distances = [] 

    for i in range(0,9):
        CC_scores = CC_test.judge([cont,list_of_instances[i]])
        v = CC_scores.score.iloc[1,0]
        ws_distances.append(v)
    return ws_distances

In [93]:
cont_rest = [f'{base}\\continuous\\res_1_8\\seed_{i}\\spike_data.pkl'.format(base,i) for i in range(2,11)]

In [94]:
cont_rest_classes = get_classes(cont_rest)

In [95]:
cont_rest_instances = create_instances(cont_rest_classes)

In [80]:
ws_cont_rest = get_ws_distances(cont_rest_instances)

In [81]:
ws_cont_rest

[0.035672880709171295,
 0.026252809911966324,
 0.05190180242061615,
 0.05124605447053909,
 0.040055740624666214,
 0.03325814753770828,
 0.03180937469005585,
 0.03627137839794159,
 0.04224839806556702]

In [96]:
ws_cont_rest_cv = get_ws_distances_cv(cont_rest_instances)

C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Nida\anaconda3\envs\nunit\lib\site-packages\numpy\core\_methods.py:262: RuntimeWa

In [99]:
ws_cont_rest_cc = get_ws_distances_cc(cont_rest_instances)